In [53]:
!pip install python-mnist

In [54]:
from mnist.loader import MNIST
import pandas as pd

In [55]:
def Accuracy(y_true, y_pred):
  corr = 0
  for num,i in enumerate(y_true):
    if i==y_pred[num]:
      corr+=1
  corr /= len(y_true)
  return corr*100

In [56]:
def loadMNIST():
    mndata = MNIST('/content/MNIST')
    imagesTest, labelsTest = mndata.load_training()
    imagesTrain, labelsTrain = mndata.load_testing()
    return imagesTrain, labelsTrain, imagesTest, labelsTest
X_train, y_train, X_test, y_test = loadMNIST()

In [57]:
from sklearn.linear_model import LogisticRegression
def Linear_Classifier_and_Pairwise_Linear_Classifier():
    model = LogisticRegression(random_state=0, max_iter=1000)
    model = model.fit(X_train, y_train)
    return Accuracy(model.predict(X_test), y_test)

In [58]:
from sklearn.neighbors import KNeighborsClassifier
def Baseline_Nearest_Neighbor_Classifer():
    # k=3 was picked from the paper 
    # sklearn's defualt is 5
    model = KNeighborsClassifier(n_neighbors=3)
    model = model.fit(X_train, y_train)
    return Accuracy(model.predict(X_test), y_test)

In [59]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
def Principal_Component_Analysis_and_Polynomial_Classifier():
    # Convert 28*28 dim to 40
    # used to train a 2nd degree polynomial Classifer
    pca = PCA(n_components=40).fit(X_train)
    poly = PolynomialFeatures(2)
    X_train_updated = poly.fit_transform(pca.transform(X_train))
    X_test_updated = poly.transform(pca.transform(X_test))
    model = LogisticRegression(random_state=0, max_iter=1000)
    model = model.fit(X_train_updated, y_train)
    return Accuracy(model.predict(X_test_updated), y_test)

In [ ]:
models = ["Linear_Classifier_and_Pairwise_Linear_Classifier", "Baseline_Nearest_Neighbor_Classifer", 
          "Principal_Component_Analysis_and_Polynomial_Classifier"]
names, acc = [], []
for i in models:
    names.append(" ".join(i.split("_")))
    acc.append(eval(i + "()"))
df = pd.DataFrame()
df["Models"] = names
df["Accuracies"] = acc

In [61]:
df

,Models,Accuracies
0,Linear Classifier and Pairwise Linear Classifier,85.881667
1,Baseline Nearest Neighbor Classifer,94.283333
2,Principal Component Analysis and Polynomial Cl...,95.143333
